In [46]:
# Detta skript inporterar en xml-fil från tex länsstyrelsernas geodatakatalog och exporterar ut valda attribut till Zotero RDF för import ill Zotero
# https://ext-geodatakatalog.lansstyrelsen.se/GeodataKatalogen/srv/api/records/ed5814b2-08bf-493a-a164-7819e1b590d6/formatters/xml

import requests                 # Nödvändigt för att hämta data via URL
from lxml import etree as ET    # Nödvändigt för att parsa XML
import datetime 
import re
from IPython.display import display, Markdown # Nödvändigt för att visa markdown/felmeddelanden

# --- Globala Hjälpfunktioner ---

def get_bounding_box(root, namespaces):
    """
    Extraherar Bounding Box-informationen från ISO 19139-XML. 
    Tar nu emot 'namespaces' som argument för säker parsning.
    """
    
    # Sök efter gmd:EX_GeographicBoundingBox
    bbox_element = root.find('.//gmd:EX_GeographicBoundingBox', namespaces)
    
    if bbox_element is None:
        return ""

    try:
        # Vi använder 'namespaces' som skickas in
        w = clean_tag(bbox_element.find('.//gco:Decimal[../gmd:westBoundLongitude]', namespaces))
        e = clean_tag(bbox_element.find('.//gco:Decimal[../gmd:eastBoundLongitude]', namespaces))
        s = clean_tag(bbox_element.find('.//gco:Decimal[../gmd:southBoundLatitude]', namespaces))
        n = clean_tag(bbox_element.find('.//gco:Decimal[../gmd:northBoundLatitude]', namespaces))
        
        if all([w, e, s, n]):
            return f"Bounding Box (Decimalgrader): W={w}, E={e}, S={s}, N={n}"
    except:
        pass
    
    return ""

In [47]:
# Block 2: NY HJÄLPFUNKTION FÖR ATT LÄSA IN DATA VIA URL (Robust URL-hantering)

def fetch_xml_data(identifier_url):
    """
    Hämtar rå ISO 19139 XML från en GeoNetwork URL.
    Accepterar olika format och konstruerar den säkraste XML-hämtnings-URL:en.
    
    Returnerar: (xml_content_string, csw_url_final_for_zotero)
    """
    
    # REGEX för att hitta GUID (UUID)
    GUID_REGEX = r'([a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12})'
    
    match = re.search(GUID_REGEX, identifier_url)

    if not match:
        raise ValueError("Kunde inte hitta GUID (UUID) i den angivna URL:en. Kontrollera formatet.")
            
    guid = match.group(1)
    
    # 1. Hämta bas-URL:en
    try:
        base_url = identifier_url.split('/srv/')[0]
    except IndexError:
        raise ValueError("URL:en verkar inte vara en GeoNetwork-URL (saknar '/srv/').")


    # 2. KONSTRUERA DE BEHÖVDA URL:ERNA
    
    # A. Den mest pålitliga URL:en för rå XML-hämtning
    xml_url = f"{base_url}/srv/api/records/{guid}/formatters/xml" 
    
    # B. Den publika URL:en som ska lagras i Zotero-posten
    csw_url_final = f"{base_url}/srv/swe/catalog.search#/metadata/{guid}"
    
    
    # 3. Hämta datan
    try:
        # Använder accept header för att säkerställa XML
        headers = {'Accept': 'application/xml, application/rdf+xml'} 
        response = requests.get(xml_url, headers=headers, timeout=10)
        response.raise_for_status() 

        # Kontrollera Content-Type för att undvika HTML-parsfel
        content_type = response.headers.get('Content-Type', '').lower()
        if 'text/html' in content_type:
             raise ConnectionError(f"Servern returnerade HTML (Content-Type: {content_type}) från {xml_url}.")
             
        # Dubbelkolla innehållet
        content_text = response.content.decode('utf-8', errors='ignore').strip()
        if content_text.startswith('<!DOCTYPE html') or content_text.startswith('<html'):
            raise ConnectionError(f"Innehållet är HTML (inte rå XML) från {xml_url}.")
             
        return response.content, csw_url_final
        
    except requests.exceptions.RequestException as e:
        raise ConnectionError(f"Kunde inte hämta XML-data från {xml_url}: {e}")

In [51]:
# Block 3: Huvudkonverteringsfunktion (iso19139_till_zotero_rdf) - SLUTLIG VERSION

import datetime 
import re
from lxml import etree as ET

# --- Lokala Hjälpfunktioner (Inbäddade) ---

def clean_tag(element):
    """Extraherar text från ett lxml-element och rensar upp."""
    return element.text if element is not None and element.text else ''

def sanitize_text_single_line(text):
    """Rensar text från inledande/avslutande blanksteg och konverterar till en rad."""
    if text is None:
        return ''
    return ' '.join(text.split()).strip()

def sanitize_text_preserve_newlines(text):
    """Rensar text men bevarar radbrytningar för abstract-fältet."""
    if text is None:
        return ''
    lines = [line.strip() for line in text.splitlines()]
    return '\n'.join(line for line in lines if line).strip()


# --- Huvudfunktion ---

def iso19139_till_zotero_rdf(xml_data, csw_url):
    """
    Parsar ISO 19139 XML och konverterar till Zotero RDF XML-struktur.
    """
    
    # --- ZOTERO NAMESPACES ---
    RDF_NS = "http://www.w3.org/1999/02/22-rdf-syntax-ns#"
    DC_NS = "http://purl.org/dc/elements/1.1/"
    ZOTERO_NS = "http://www.zotero.org/namespaces/export#" 
    FOAF_NS = "http://xmlns.com/foaf/0.1/"
    BIB_NS = "http://purl.org/net/biblio#"
    DCTERMS_NS = "http://purl.org/dc/terms/"
    LINK_NS = "http://purl.org/rss/1.0/modules/link/" 
    
    # --- NAMNRYMDER FÖR XML-PARSNING (ISO 19139) ---
    ISO_NAMESPACES = {
        'gmd': 'http://www.isotc211.org/2005/gmd',
        'gco': 'http://www.isotc211.org/2005/gco',
        'gml': 'http://www.opengis.net/gml',
        'srv': 'http://www.isotc211.org/2005/srv'
    }
    
    try:
        root = ET.fromstring(xml_data)
    except Exception as e:
        raise ValueError(f"FEL vid parsning av XML med lxml: {e}")
        
    # --- Datahämtning och Sanering (ANVÄNDER ROBUST XPATH) ---
    
    # Titel
    xpath_titel = './/*[local-name()="MD_DataIdentification"]/*[local-name()="citation"]/*[local-name()="CI_Citation"]/*[local-name()="title"]/*[local-name()="CharacterString"]'
    titel_element = root.xpath(xpath_titel, namespaces=ISO_NAMESPACES)
    titel = clean_tag(titel_element[0]) if titel_element else ''
    
    # Abstrakt
    xpath_abstrakt = './/*[local-name()="MD_DataIdentification"]/*[local-name()="abstract"]/*[local-name()="CharacterString"]'
    abstrakt_element = root.xpath(xpath_abstrakt, namespaces=ISO_NAMESPACES)
    abstrakt = clean_tag(abstrakt_element[0]) if abstrakt_element else ''

    # Datum (FÖRBÄTTRAD FELHANTERING HÄR)
    latest_date = None
    
    xpath_date_types = './/gmd:CI_Date[gmd:dateType/gmd:CI_DateTypeCode/@codeListValue="creation" or gmd:dateType/gmd:CI_DateTypeCode/@codeListValue="revision"]/gmd:date/gco:Date'
    
    date_elements = root.xpath(xpath_date_types, namespaces=ISO_NAMESPACES)
    
    for date_el in date_elements:
        date_str = clean_tag(date_el)
        # Försök parsa både YYYY-MM-DD och enklare YYYY
        try:
            current_date = datetime.datetime.strptime(date_str, '%Y-%m-%d').date()
        except ValueError:
            try:
                current_date = datetime.datetime.strptime(date_str, '%Y').date()
            except ValueError:
                continue # Hoppa över om datumet inte kunde parsas

        if latest_date is None or current_date > latest_date:
            latest_date = current_date
            
    # Formatera output som YYYY-MM-DD eller YYYY-01-01 som fallback
    if latest_date:
        date_output_str = latest_date.strftime("%Y-%m-%d")
    else:
        # Säker fallback, använder dagens datum om inget hittas
        date_output_str = datetime.date.today().strftime("%Y-%m-%d") 
    
    # Ansvarig organisation
    ansvarig_list = []
    xpath_responsible = './/*[local-name()="CI_ResponsibleParty"]/*[local-name()="role"]/*[local-name()="CI_RoleCode"][@codeListValue="owner" or @codeListValue="custodian"]/../../*[local-name()="organisationName"]/*[local-name()="CharacterString"]'
    
    for org_element in root.xpath(xpath_responsible, namespaces=ISO_NAMESPACES):
        ansvarig_list.append(clean_tag(org_element))
        
    if not ansvarig_list:
        ansvarig_list.append('Geodata Provider') 

    # Hämta Spatial Representation Type (för Zotero Medium)
    xpath_spatial_format = './/gmd:spatialRepresentationType/gmd:MD_SpatialRepresentationTypeCode/@codeListValue'
    spatial_format_element = root.xpath(xpath_spatial_format, namespaces=ISO_NAMESPACES)
    spatial_format = sanitize_text_single_line(spatial_format_element[0]) if spatial_format_element else ''
    
    # GUID
    match = re.search(r'([a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12})', csw_url)
    guid = match.group(1) if match else 'no-guid-found'
    
    # Hämta BBox
    bbox_info = get_bounding_box(root, ISO_NAMESPACES) 

    titel_clean = sanitize_text_single_line(titel)
    license_clean = 'Creative Commons CC0 1.0 Public Domain Dedication' 
    
    # KORRIGERAD TAGGLOGIK
    tags_list = ["GIS", "geodata"] 
    
    xpath_keywords = './/*[local-name()="MD_Keywords"]/*[local-name()="keyword"]/*[local-name()="CharacterString"]'
    for tag_element in root.xpath(xpath_keywords, namespaces=ISO_NAMESPACES):
        tag = sanitize_text_single_line(clean_tag(tag_element))
        if tag and tag not in tags_list:
             tags_list.append(tag)

    abstract_content = sanitize_text_preserve_newlines(abstrakt)
    if bbox_info:
        abstract_content += f"\n\n--- Geografisk Utbredning ---\n{sanitize_text_preserve_newlines(bbox_info)}"
    
    # --- 3. BYGG ZOTERO RDF XML ---
    
    # Definiera namespace-mappning för rot-elementet
    ns_map_list = [
        ('rdf', RDF_NS), ('z', ZOTERO_NS), ('dc', DC_NS), 
        ('foaf', FOAF_NS), ('bib', BIB_NS), ('link', LINK_NS), 
        ('dcterms', DCTERMS_NS)
    ]
    ns_map = dict(ns_map_list)
    
    # Säkerställer att rdf_root skapas
    rdf_root = ET.Element(f"{{{RDF_NS}}}RDF", nsmap=ns_map)
    
    # --- A. HUVUDPOST ---
    
    link_id = guid + "-link"
    
    item_element = ET.SubElement(rdf_root, f"{{{RDF_NS}}}Description", 
                                 attrib={f"{{{RDF_NS}}}about": csw_url}) 
    
    ET.SubElement(item_element, f"{{{ZOTERO_NS}}}itemType").text = "dataset"
    
    # Skapare (Organisationer)
    if ansvarig_list:
        authors_element = ET.SubElement(item_element, f"{{{BIB_NS}}}authors")
        seq_element = ET.SubElement(authors_element, f"{{{RDF_NS}}}Seq") 
        for org in ansvarig_list:
            li_element = ET.SubElement(seq_element, f"{{{RDF_NS}}}li")
            
            org_element = ET.SubElement(li_element, f"{{{FOAF_NS}}}Person")
            ET.SubElement(org_element, f"{{{FOAF_NS}}}surname").text = sanitize_text_single_line(org)

    # Länk, Tags, Titel, Abstract, Datum, Identifier, Licens
    ET.SubElement(item_element, f"{{{LINK_NS}}}link", 
                  attrib={f"{{{RDF_NS}}}resource": f"#{link_id}"}) 

    for tag in tags_list:
        ET.SubElement(item_element, f"{{{DC_NS}}}subject").text = tag
        
    ET.SubElement(item_element, f"{{{DC_NS}}}title").text = titel_clean
    ET.SubElement(item_element, f"{{{DCTERMS_NS}}}abstract").text = abstract_content 
    ET.SubElement(item_element, f"{{{DC_NS}}}date").text = date_output_str 
    
    identifier_element = ET.SubElement(item_element, f"{{{DC_NS}}}identifier")
    uri_element = ET.SubElement(identifier_element, f"{{{DCTERMS_NS}}}URI")
    ET.SubElement(uri_element, f"{{{RDF_NS}}}value").text = csw_url
    
    ET.SubElement(item_element, f"{{{DC_NS}}}rights").text = license_clean 
    
    # --- ZOTERO-SPECIFIKA FÄLT ---
    
    # 1. <z:type>Geodata</z:type>
    ET.SubElement(item_element, f"{{{ZOTERO_NS}}}type").text = "Geodata"
    
    # 2. <z:medium>vector</z:medium>
    if spatial_format:
        ET.SubElement(item_element, f"{{{ZOTERO_NS}}}medium").text = spatial_format
    
    # --- B. BILAGA (ATTACHMENT) ---
    
    attachment_element = ET.SubElement(rdf_root, f"{{{ZOTERO_NS}}}Attachment", 
                                        attrib={f"{{{RDF_NS}}}about": f"#{link_id}"})
    ET.SubElement(attachment_element, f"{{{ZOTERO_NS}}}itemType").text = "attachment"
    ET.SubElement(attachment_element, f"{{{DC_NS}}}title").text = "Kataloglänk"
    
    att_identifier_element = ET.SubElement(attachment_element, f"{{{DC_NS}}}identifier")
    att_uri_element = ET.SubElement(att_identifier_element, f"{{{DCTERMS_NS}}}URI")
    ET.SubElement(att_uri_element, f"{{{RDF_NS}}}value").text = csw_url

    ET.SubElement(attachment_element, f"{{{ZOTERO_NS}}}linkMode").text = "1" 
    
    # SLUTLIGT STEG: Kontrollera att utdatan är en sträng
    xml_output = ET.tostring(rdf_root, encoding='utf-8', pretty_print=True, xml_declaration=False).decode('utf-8')

    return xml_output

In [55]:
# Block 4: Utför konvertering och spara (MED ANVÄNDARINPUT & SÄKERT FILNAMN)

# Steg 1: Begär input-URL från användaren
input_url = input("Vänligen mata in URL (Kataloglänk, API-sök, Rå XML-länk, etc.):\n")

try:
    # Steg 2: Hämta XML-data och den publika katalogsökvägen
    xml_data_content, csw_url_final = fetch_xml_data(input_url)
    
    # Steg 3: Utför konvertering
    rdf_output = iso19139_till_zotero_rdf(xml_data_content, csw_url_final) 

    # --- FILNAMNSFIX: Extrahera GUID för ett säkert filnamn ---
    guid_match = re.search(r'([a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12})', csw_url_final)
    
    if guid_match:
        guid = guid_match.group(1)
    else:
        # Fallback till ett säkert, generiskt namn om GUID inte kan hittas
        guid = datetime.datetime.now().strftime("%Y%m%d%H%M%S") 
    
    output_filename = f"zotero_metadata_{guid}.rdf"
    # --- SLUT PÅ FILNAMNSFIX ---
    
    # Steg 4: Skriv till .rdf-fil
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write(rdf_output)
    
    display(Markdown(f"**Slutfört!** Filen **`{output_filename}`** har skapats.\n\n"
                     f"Importera denna fil till Zotero."))
    
except (ValueError, ConnectionError) as e:
    # Fångar fel från parsning (Block 3) och hämtning (Block 2)
    display(Markdown(f"**FEL vid hämtning/parsning:** {e}"))
except Exception as e:
    # Fångar andra fel (t.ex. filskrivningsfel)
    display(Markdown(f"**FEL:** Kunde inte skriva till fil eller konvertera: {e}"))

Vänligen mata in URL (Kataloglänk, API-sök, Rå XML-länk, etc.):
 https://www.geodata.se/geodataportalen/srv/swe/catalog.search#/metadata/8dc153f5-a26f-40c7-b735-fbe4f1278815


**Slutfört!** Filen **`zotero_metadata_8dc153f5-a26f-40c7-b735-fbe4f1278815.rdf`** har skapats.

Importera denna fil till Zotero.